<a href="https://colab.research.google.com/github/abilincoln1/FP-Markets-AI-Trader-/blob/main/EV.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
with open('/content/deepseek_python_20250818_540d41.py', 'r') as f:
    print(f.read())

# %% [markdown]
# # 🚛 EV Fleet Battery Failure Predictor (UK-Optimized)
# *Train an edge-compatible model to predict battery failures 24hrs in advance*

# %% [markdown]
# ## Step 1: Install Libraries
!pip install pandas scikit-learn tensorflow tflite-support obd-py pyarrow -q

# %% [markdown]
# ## Step 2: Load UK Fleet Data (Sample Dataset)
import pandas as pd
from google.colab import files

# Upload your OBD-II logs (or use our sample)
uploaded = files.upload()
df = pd.read_parquet(next(iter(uploaded.keys())))

# Sample synthetic data if no upload
if df.empty:
  print("Using synthetic UK data...")
  data = {
      'voltage': [13.2, 12.8, 14.1, 11.9, 13.5],
      'temp_c': [18, 22, -3, 35, 19],  # UK weather range
      'charge_cycles': [120, 345, 87, 500, 210],
      'failed_next_24hrs': [0, 1, 0, 1, 0]  # Target
  }
  df = pd.DataFrame(data)

# %% [markdown]
# ## Step 3: Train Lightweight Model (Edge Optimized)
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_se

In [ ]:
# %% [markdown]
# # 🚛 EV Fleet Battery Failure Predictor (UK-Optimized)
# *Train an edge-compatible model to predict battery failures 24hrs in advance*

# %% [markdown]
# ## Step 1: Install Libraries
!pip install pandas scikit-learn tensorflow tflite-support obd-py pyarrow -q

# %% [markdown]
# ## Step 2: Load UK Fleet Data (Sample Dataset)
import pandas as pd
from google.colab import files

# Upload your OBD-II logs (or use our sample)
uploaded = files.upload()
df = pd.read_parquet(next(iter(uploaded.keys())))

# Sample synthetic data if no upload
if df.empty:
  print("Using synthetic UK data...")
  data = {
      'voltage': [13.2, 12.8, 14.1, 11.9, 13.5],
      'temp_c': [18, 22, -3, 35, 19],  # UK weather range
      'charge_cycles': [120, 345, 87, 500, 210],
      'failed_next_24hrs': [0, 1, 0, 1, 0]  # Target
  }
  df = pd.DataFrame(data)

# %% [markdown]
# ## Step 3: Train Lightweight Model (Edge Optimized)
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

# Feature engineering
df['voltage_drop'] = df['voltage'].rolling(3).std().fillna(0)
df['cold_weather_risk'] = (df['temp_c'] < 5).astype(int)

X = df[['voltage', 'temp_c', 'charge_cycles', 'voltage_drop', 'cold_weather_risk']]
y = df['failed_next_24hrs']

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train model
model = RandomForestClassifier(n_estimators=50, max_depth=5)
model.fit(X_train, y_train)

# Evaluate
print(f"Accuracy: {model.score(X_test, y_test):.2f}")

# %% [markdown]
# ## Step 4: Convert to TensorFlow Lite (For OBD-II Dongles)
import tensorflow as tf
from sklearn.tensorflow import export_saved_model

# Convert sklearn → TF → TFLite
tf_model = export_saved_model(model, "saved_model")
converter = tf.lite.TFLiteConverter.from_saved_model("saved_model")
tflite_model = converter.convert()

# Download for edge devices
with open('battery_predictor.tflite', 'wb') as f:
  f.write(tflite_model)
files.download('battery_predictor.tflite')

print("✅ TFLite model ready for Raspberry Pi/OBD-II!")

ERROR: Could not find a version that satisfies the requirement obd-py (from versions: none)
ERROR: No matching distribution found for obd-py
